In [ ]:
import openai,re
from utils import ChatSession,update_investor_profile
import pandas as pd

# load and set our key
openai.api_key = open("key.txt", "r").read().strip("\n")

# questions:
#         'Is the Customer older than 50? Say yes or no.',\
#         'Does the Customer earn less than 100K annually? Say yes or no.',\
#         'Does the Customer have a high risk appetite? Say yes or no.'

RuleBasedPortfolios = pd.read_excel('RuleBasedPortfolios.xlsx')
RuleBasedPortfolios.columns = RuleBasedPortfolios.columns.map(lambda x: x.lower())

RuleBasedPortfolios.age = RuleBasedPortfolios.age.apply(lambda x: 'yes' if x == '50 -' else 'no') # use .strip()
RuleBasedPortfolios.income = RuleBasedPortfolios.income.apply(lambda x: 'yes' if x == '0 - 100' else 'no') # use .strip()
RuleBasedPortfolios['risk appetite'] = RuleBasedPortfolios['risk appetite'].apply(lambda x: 'yes' if x == 'High' else 'no') # use .strip()

# GPT/GPT

In [ ]:
session1 = ChatSession(gpt_name='Advisor')
session2 = ChatSession(gpt_name='Customer')

beginning_sentence = "Hi. I am not sure about how to invest. Could you help me?"
# beginning_sentence = "Hi. I am curious about today's investment chances. Could you help me?"

session2.inject(line="You are a customer at a bank, seeking financial advise from me. You will ask general questions about current financial situation in the world and Switzerland. You are 50 years old with an income of 120K. Your risk appetite is low. Do not give these information unless you are asked for. Say ok if you understand.",role="user")
session2.inject(line="Ok.",role= "assistant")
session2.inject(line=beginning_sentence,role= "assistant")

print('Customer: ' + beginning_sentence)

session1.inject(line="You are a financial advisor at a bank. You should ask about customers' age, income and risk apetite later in the conversation before suggesting a portfolio. Be subtle about asking for these information and do not ask at the very beginning of the conversation. Always prioritize answering the customers' questions over asking for these information. I am a customer seeking financial advise from you. Say ok if you understand.",role="user")
session1.inject(line="Ok.",role= "assistant")

investor_profile = {i:False for i in ['age','income','risk appetite']}
while True:
    session1.chat(user_input=session2.messages[-1],temperature=0.01)
    session2.chat(user_input=session1.messages[-1])
    update_investor_profile(investor_profile=investor_profile,dialogue=f'{session1.gpt_name}: {session1.messages[-1].content}'+'\n'+f'{session2.gpt_name}: {session2.messages[-1].content}')

    
    if not len([i for i in investor_profile.values() if not i]):
        assert sum(map(lambda x: 'no' in x.lower() or 'yes' in x.lower(),investor_profile.values())) == len(investor_profile.keys()), investor_profile
        break

# Rule based portfolio by using ``investor_profile``
portfolio = RuleBasedPortfolios.where(lambda x: x['age'].apply(lambda y: y in investor_profile['age'].lower())*\
                            x['income'].apply(lambda y: y in investor_profile['income'].lower())*\
                                x['risk appetite'].apply(lambda y: y in investor_profile['risk appetite'].lower()))['portfolio'].dropna().values
assert portfolio.size == 1
portfolio = int(portfolio.item())

# Tell GPT to recommend portfolio
session1.inject(line=f"Then, I would recommend portfolio {portfolio}.",role= "assistant")
session1(1)


# GPT/Human

In [ ]:
session1 = ChatSession(gpt_name='Advisor')

user_input = input("> ")
print('Customer: ', user_input)

session1.inject(line="You are a financial advisor at a bank. You should ask about customers' age, income and risk apetite later in the conversation before suggesting a portfolio. Be subtle about asking for these information and do not ask at the very beginning of the conversation. Always prioritize answering the customers' questions over asking for these information. I am a customer seeking financial advise from you. Say ok if you understand.",role="user")
session1.inject(line="Ok.",role= "assistant")

investor_profile = {i:False for i in ['age','income','risk appetite']}
pattern = re.compile(r'\w+')
while True:
    session1.chat(user_input=user_input,verbose=False)
    print('Advisor: ', session1.messages[-1].content)
    user_input = input("> ")
    print('Customer: ', user_input)
    if re.search(pattern,user_input.strip()) is not None:
        update_investor_profile(investor_profile=investor_profile,dialogue=f'{session1.gpt_name}: {session1.messages[-1].content}'+'\n'+f'Customer: {user_input}')
    if not len([i for i in investor_profile.values() if not i]):
        break

# Rule based portfolio by using ``investor_profile``
portfolio = RuleBasedPortfolios.where(lambda x: x['age'].apply(lambda y: y in investor_profile['age'].lower())*\
                            x['income'].apply(lambda y: y in investor_profile['income'].lower())*\
                                x['risk appetite'].apply(lambda y: y in investor_profile['risk appetite'].lower()))['portfolio'].dropna().values
assert portfolio.size == 1
portfolio = int(portfolio.item())

# Tell GPT to recommend portfolio
session1.inject(line=f"Then, I would recommend portfolio {portfolio}.",role= "assistant")
session1(1)